In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install bayesian-optimization
!pip install scikit-optimize
import h5py
import numpy as np
import tensorflow as tf
import sklearn.metrics as metrics
import itertools
import matplotlib.pylab as plt
import skopt
import pandas as pd

from tensorflow.keras import layers, models
from tensorflow.python.keras import backend as K
from tensorflow.python.framework import ops
from skopt import gp_minimize
from skopt.space import Categorical, Integer
from skopt.utils import use_named_args
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
from itertools import cycle
from scipy import interp
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size

In [ ]:
%cd ... # <--- INSERT HERE (...) THE PATH OF THE FOLDER

In [ ]:
ds_train_h5f = h5py.File("./train-zyx-250x190x270.h5",'r')
ds_val_h5f = h5py.File("./val-zyx-250x190x270.h5",'r')
ds_test_h5f = h5py.File("./test-zyx-250x190x270.h5",'r')

X_train = ds_train_h5f["train_X"]
Y_train = ds_train_h5f["train_Y"]

X_val = ds_val_h5f["val_X"]
Y_val = ds_val_h5f["val_Y"]

X_test = ds_test_h5f["test_X"]
Y_test = ds_test_h5f["test_Y"]

#print(X_train.shape)
#print(Y_train.shape)

#print(X_val.shape)
#print(Y_val.shape)

#print(X_test.shape)
#print(Y_test.shape)

idx_map_train = np.arange(X_train.shape[0])
np.random.shuffle(idx_map_train)

idx_map_val = np.arange(X_val.shape[0])
np.random.shuffle(idx_map_val)

idx_map_test = np.arange(X_test.shape[0])
np.random.shuffle(idx_map_test)

In [ ]:
def generate_batches_from_train_hdf5_file(hdf5_file, batch_size, idx_map):
  file_size = len(hdf5_file['train_Y'])

  while 1:
    # Count how many entries we have read
    n_entries = 0
    # As long as we haven't read all entries from the file: keep reading
    while n_entries < (file_size - batch_size):
      # Start the next batch at index 0
      # Create numpy arrays of input data (features)
      xs = hdf5_file['train_X'][n_entries: n_entries + batch_size,:,:,:]
      xs = np.array(xs)
      xs = xs[..., np.newaxis]

      ys = hdf5_file['train_Y'][n_entries:n_entries + batch_size]
      ys = np.array(ys)

      # We have read one more batch from this file
      n_entries += batch_size
      yield (xs, ys)

In [ ]:
def generate_batches_from_val_hdf5_file(hdf5_file, batch_size, idx_map):
  file_size = len(hdf5_file['val_Y'])

  while 1:
    # Count how many entries we have read
    n_entries = 0
    # As long as we haven't read all entries from the file: keep reading
    while n_entries < (file_size - batch_size):
      # Start the next batch at index 0
      # Create numpy arrays of input data (features)
      xs = hdf5_file['val_X'][n_entries: n_entries + batch_size,:,:,:]
      xs = np.array(xs)
      xs = xs[..., np.newaxis]

      ys = hdf5_file['val_Y'][n_entries:n_entries + batch_size]
      ys = np.array(ys)

      # We have read one more batch from this file
      n_entries += batch_size
      yield (xs, ys)

In [ ]:
def generate_batches_from_test_hdf5_file(hdf5_file, batch_size, idx_map):
  file_size = len(hdf5_file['test_Y'])

  while 1:
    # Count how many entries we have read
    n_entries = 0
    # As long as we haven't read all entries from the file: keep reading
    while n_entries < (file_size - batch_size):
      # Start the next batch at index 0
      # Create numpy arrays of input data (features)
      xs = hdf5_file['test_X'][n_entries: n_entries + batch_size,:,:,:]
      xs = np.array(xs)
      xs = xs[..., np.newaxis]

      ys = hdf5_file['test_Y'][n_entries:n_entries + batch_size]
      ys = np.array(ys)

      # We have read one more batch from this file
      n_entries += batch_size
      yield (xs, ys)

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues):
  plt.figure(figsize=(6, 6), dpi=80)

  im = plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)

  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, round(cm[i, j],2),
      horizontalalignment="center",
      color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')

  ax = plt.gca()
  divider = make_axes_locatable(ax)
  cax = divider.append_axes("right", size="5%", pad=0.05)
  plt.colorbar(im, cax=cax)

In [ ]:
def getConvLSTMModel(my_dropout_rate1,
                     my_dropout_rate2,
                     my_dropout_rate3,
                     my_learning_rate,
                     my_batch_size,
                     verbose=True):

  model = models.Sequential()
  model.add(layers.ConvLSTM2D(filters=8,
                              kernel_size=(3, 3),
                              input_shape=(250, 190, 270, 1),
                              name="convlstm2d_1"))
  model.add(layers.Dropout(my_dropout_rate1,
                           name="dropout_1"))
  model.add(layers.Flatten(name="flatten_1"))
  model.add(layers.Dense(256,
                         activation="relu",
                         name="dense_1"))
  model.add(layers.Dropout(my_dropout_rate2,
                           name="dropout_2"))
  model.add(layers.Dense(128,
                         activation="relu",
                         name="dense_2"))
  model.add(layers.Dropout(my_dropout_rate3,
                           name="dropout_3"))
  model.add(layers.Dense(2,
                         activation='softmax',
                         name="dense_3"))
  if verbose:
    model.summary()

  opt = tf.keras.optimizers.SGD(learning_rate=my_learning_rate)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

  return model

In [ ]:
dim_dropout1_rate = Categorical([0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8], name='my_dropout_rate1')
dim_dropout2_rate = Categorical([0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8], name='my_dropout_rate2')
dim_dropout3_rate = Categorical([0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8], name='my_dropout_rate3')
dim_learning_rate = Categorical([0.0001, 0.0005, 0.001, 0.005], name='my_learning_rate')
dim_batch_size = Integer(low=1, high=5, name="my_batch_size")

dimensions = [dim_dropout1_rate,
              dim_dropout2_rate,
              dim_dropout3_rate,
              dim_learning_rate,
              dim_batch_size]

default_parameters = [0.5, 0.5, 0.5, 0.001, 1]

In [ ]:
@use_named_args(dimensions=dimensions)

def fitness(my_dropout_rate1,my_dropout_rate2,my_dropout_rate3,my_learning_rate,my_batch_size):

  model = getConvLSTMModel(my_dropout_rate1=my_dropout_rate1,
                           my_dropout_rate2=my_dropout_rate2,
                           my_dropout_rate3=my_dropout_rate3,
                           my_learning_rate=my_learning_rate,
                           my_batch_size=my_batch_size)

  generator_train = generate_batches_from_train_hdf5_file(ds_train_h5f, 1, idx_map_train)
  generator_val = generate_batches_from_val_hdf5_file(ds_val_h5f, 1, idx_map_val)

  # Named blackbox because it represents the structure
  earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=3, verbose=1, restore_best_weights=True)
  callbacks = [earlystop]

  blackbox = model.fit(x=generator_train,
                      epochs=15,
                      batch_size=my_batch_size,
                      validation_data=generator_val,
                      steps_per_epoch=int(720//my_batch_size),
                      callbacks=callbacks,
                      validation_steps=int(50//my_batch_size))

  # Return the validation accuracy for the last epoch
  print(blackbox.history.keys())
  accuracy = blackbox.history['val_accuracy'][-1]

  # Print the classification accuracy
  print()
  print("Accuracy: {0:.2%}".format(accuracy))
  print()

  # Delete the Keras model with these hyper-parameters from memory
  del model

  # Clear the Keras session, otherwise it will keep adding new models to the same TensorFlow graph each time we create a model with a different set of hyper-parameters
  K.clear_session()
  ops.reset_default_graph()

  return -accuracy

In [ ]:
gp_result = gp_minimize(func=fitness,
                        dimensions=dimensions,
                        noise='gaussian', # set this to a value close to zero (1e-10) if the function is noise-free (default is 'gaussian')
                        n_jobs=1, # if n_jobs=-1 number of jobs is set to number of cores (default is 1)
                        n_calls=11, # int (default is 100)
                        kappa=1.96, # (default is 1.96)
                        x0=default_parameters)

In [ ]:
print("The best accuracy was: "+str(round(gp_result.fun *-100,2))+"%")

In [ ]:
gp_result.x

In [ ]:
pd.concat([pd.DataFrame(gp_result.x_iters, columns = ["my_dropout_rate1","my_dropout_rate2","my_dropout_rate3","my_learning_rate","my_batch_size"]),
(pd.Series(gp_result.func_vals*-100, name="accuracy"))], axis=1)

In [ ]:
### 1 ###

def getConvLSTMModel(verbose=True):

  model = models.Sequential()
  model.add(layers.ConvLSTM2D(filters=8,
                              kernel_size=(3, 3),
                              input_shape=(250, 190, 270, 1),
                              name="convlstm2d_1"))
  model.add(layers.Dropout(0.7,
                           name="dropout_1"))
  model.add(layers.Flatten(name="flatten_1"))
  model.add(layers.Dense(256,
                         activation="relu",
                         name="dense_1"))
  model.add(layers.Dropout(0.3,
                           name="dropout_2"))
  model.add(layers.Dense(128,
                         activation="relu",
                         name="dense_2"))
  model.add(layers.Dropout(0.3,
                           name="dropout_3"))
  model.add(layers.Dense(2,
                         activation='softmax',
                         name="dense_3"))
  if verbose:
    model.summary()

  opt = tf.keras.optimizers.SGD(learning_rate=0.001)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

  return model

In [ ]:
### 2 ###

def getConvLSTMModel(verbose=True):

  model = models.Sequential()
  model.add(layers.ConvLSTM2D(filters=8,
                              kernel_size=(3, 3),
                              input_shape=(250, 190, 270, 1),
                              name="convlstm2d_1"))
  model.add(layers.Dropout(0.5,
                           name="dropout_1"))
  model.add(layers.Flatten(name="flatten_1"))
  model.add(layers.Dense(128,
                         activation="relu",
                         name="dense_1"))
  model.add(layers.Dropout(0.5,
                           name="dropout_2"))
  model.add(layers.Dense(2,
                         activation='softmax',
                         name="dense_2"))
  if verbose:
    model.summary()

  opt = tf.keras.optimizers.SGD(learning_rate=0.001)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

  return model

In [ ]:
### 3 ###

def getConvLSTMModel(verbose=True):

  model = models.Sequential()
  model.add(layers.ConvLSTM2D(filters=8,
                              kernel_size=(3, 3),
                              input_shape=(250, 190, 270, 1),
                              name="convlstm2d_1"))
  model.add(layers.Dropout(0.7,
                           name="dropout_1"))
  model.add(layers.Flatten(name="flatten_1"))
  model.add(layers.Dense(128,
                         activation="relu",
                         name="dense_1"))
  model.add(layers.Dropout(0.3,
                           name="dropout_2"))
  model.add(layers.Dense(2,
                         activation='softmax',
                         name="dense_2"))
  if verbose:
    model.summary()

  opt = tf.keras.optimizers.SGD(learning_rate=0.001)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

  return model

In [ ]:
model = getConvLSTMModel(True)

In [ ]:
my_batch_size = 1
my_steps_per_epoch = int(720//my_batch_size)
my_validation_steps = int(50//my_batch_size)
my_epochs = 50
my_callbacks = [
                tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=5, verbose=1, restore_best_weights=True),
                tf.keras.callbacks.ModelCheckpoint(#filepath="./model-convlstm-1.h5",
                                                   #filepath="./model-convlstm-2.h5",
                                                   #filepath="./model-convlstm-3.h5",
                                                   save_best_only=True,
                                                   mode='min',
                                                   monitor='val_loss'),
]

generator_train = generate_batches_from_train_hdf5_file(ds_train_h5f, 1, idx_map_train)
generator_val = generate_batches_from_val_hdf5_file(ds_val_h5f, 1, idx_map_val)

history = model.fit(generator_train, validation_data=generator_val,
                    steps_per_epoch=my_steps_per_epoch, validation_steps=my_validation_steps, epochs=my_epochs,
                    callbacks=my_callbacks)

In [ ]:
### 1 ###

classes = ['LUAD','LUSC']
my_batch_size = 1
my_test_steps = int(60//my_batch_size)

generator_test = generate_batches_from_test_hdf5_file(ds_test_h5f, 1, idx_map_test)

model_best = tf.keras.models.load_model("./model-convlstm-1.h5")

print("Evaluation on test data...")
loss, accuracy = model_best.evaluate(X_test, Y_test, steps=my_test_steps)
print("Loss: ",loss)
print("Accuracy: ",accuracy)

print('Prediction on test data...')
probs = model_best.predict(X_test, steps=my_test_steps)
print(probs)
#np.savetxt("./" + f'prediction-convlstm-1.csv',
#            probs,
#            delimiter=',',
#            fmt='%1.3f',
#            header=f'Prediction on NSCLC-Radiomics-Genomics test data:')
Y_test = Y_test
print(Y_test)
#np.savetxt("./" + f'ground-truth-convlstm-1.csv',
#            Y_test,
#            delimiter=',',
#            fmt='%1.3f',
#            header=f'Ground truth:')
y_pred = np.argmax(probs, axis=1)
Y_test = np.argmax(Y_test, axis=1)
print(classification_report(Y_test, y_pred, target_names=classes))

cnf_matrix = confusion_matrix(Y_test, y_pred)
cnf_matrix_norm = (cnf_matrix / cnf_matrix.astype(np.float).sum(axis=1, keepdims=True))*100
plot_confusion_matrix(cnf_matrix_norm, classes=classes, title='Confusion matrix')
plt.show()

preds = probs[:,1]
fpr, tpr, threshold = roc_curve(Y_test, preds, pos_label=1)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=2, color='b', label = 'AUC = %0.4f' % roc_auc, alpha=.8)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False positive rate (1-specificity)', fontsize=18)
plt.ylabel('True positive rate (sensitivity)', fontsize=18)
plt.title('ROC', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
### 2 ###

classes = ['LUAD','LUSC']
my_batch_size = 1
my_test_steps = int(60//my_batch_size)

generator_test = generate_batches_from_test_hdf5_file(ds_test_h5f, 1, idx_map_test)

model_best = tf.keras.models.load_model("./model-convlstm-2.h5")

print("Evaluation on test data...")
loss, accuracy = model_best.evaluate(X_test, Y_test, steps=my_test_steps)
print("Loss: ",loss)
print("Accuracy: ",accuracy)

print('Prediction on test data...')
probs = model_best.predict(X_test, steps=my_test_steps)
print(probs)
#np.savetxt("./" + f'prediction-convlstm-2.csv',
#            probs,
#            delimiter=',',
#            fmt='%1.3f',
#            header=f'Prediction on NSCLC-Radiomics-Genomics test data:')
Y_test = Y_test
print(Y_test)
#np.savetxt("./" + f'ground-truth-convlstm-2.csv',
#            Y_test,
#            delimiter=',',
#            fmt='%1.3f',
#            header=f'Ground truth:')
y_pred = np.argmax(probs, axis=1)
Y_test = np.argmax(Y_test, axis=1)
print(classification_report(Y_test, y_pred, target_names=classes))

cnf_matrix = confusion_matrix(Y_test, y_pred)
cnf_matrix_norm = (cnf_matrix / cnf_matrix.astype(np.float).sum(axis=1, keepdims=True))*100
plot_confusion_matrix(cnf_matrix_norm, classes=classes, title='Confusion matrix')
plt.show()

preds = probs[:,1]
fpr, tpr, threshold = roc_curve(Y_test, preds, pos_label=1)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=2, color='b', label = 'AUC = %0.4f' % roc_auc, alpha=.8)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False positive rate (1-specificity)', fontsize=18)
plt.ylabel('True positive rate (sensitivity)', fontsize=18)
plt.title('ROC', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
### 3 ###

classes = ['LUAD','LUSC']
my_batch_size = 1
my_test_steps = int(60//my_batch_size)

generator_test = generate_batches_from_test_hdf5_file(ds_test_h5f, 1, idx_map_test)

model_best = tf.keras.models.load_model("./model-convlstm-3.h5")

print("Evaluation on test data...")
loss, accuracy = model_best.evaluate(X_test, Y_test, steps=my_test_steps)
print("Loss: ",loss)
print("Accuracy: ",accuracy)

print('Prediction on test data...')
probs = model_best.predict(X_test, steps=my_test_steps)
print(probs)
#np.savetxt("./" + f'prediction-convlstm-3.csv',
#            probs,
#            delimiter=',',
#            fmt='%1.3f',
#            header=f'Prediction on NSCLC-Radiomics-Genomics test data:')
Y_test = Y_test
print(Y_test)
#np.savetxt("./" + f'ground-truth-convlstm-3.csv',
#            Y_test,
#            delimiter=',',
#            fmt='%1.3f',
#            header=f'Ground truth:')
y_pred = np.argmax(probs, axis=1)
Y_test = np.argmax(Y_test, axis=1)
print(classification_report(Y_test, y_pred, target_names=classes))

cnf_matrix = confusion_matrix(Y_test, y_pred)
cnf_matrix_norm = (cnf_matrix / cnf_matrix.astype(np.float).sum(axis=1, keepdims=True))*100
plot_confusion_matrix(cnf_matrix_norm, classes=classes, title='Confusion matrix')
plt.show()

preds = probs[:,1]
fpr, tpr, threshold = roc_curve(Y_test, preds, pos_label=1)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=2, color='b', label = 'AUC = %0.4f' % roc_auc, alpha=.8)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('FP rate', fontsize=18)
plt.ylabel('TP rate', fontsize=18)
plt.title('ROC curve', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()